In [1]:
import sys
sys.path.insert(0,'/home/caldaz/module/pyGSM')
from molecule import Molecule
from pes import PES
from avg_pes import Avg_PES
import numpy as np
from nifty import pvec1d,pmat2d
import matplotlib
import matplotlib.pyplot as plt
from pytc import *
import manage_xyz
from rhf_lot import *
from psiw import *
from nifty import getAllCoords,getAtomicSymbols,click,printcool
import pybel as pb
%matplotlib inline

In [2]:
printcool("Build resources")
resources = ls.ResourceList.build()
printcool('{}'.format(resources))

#========================================================#
#|                   Build resources                    |#
#========================================================#
#==============================================================#
#|                       ResourceList:                        |#
#|                       CPU threads:   1                     |#
#|                       GPU contexts:  1                     |#
#|                     GPU context details:                   |#
#|    N  ID                 Name  CC  CPU Buffer  GPU Buffer  |#
#|    0   0    GeForce GTX TITAN 3.5        1024        1024  |#
#|                                                            |#
#==============================================================#


'----------------------------------------------------------------\n'

In [3]:
printcool("build the Lightspeed (pyTC) objecs")

filepath='../data/twisted_fluoroethene.xyz'
nocc=11

molecule = ls.Molecule.from_xyz_file(filepath)
geom = geometry.Geometry.build(
    resources=resources,
    molecule=molecule,
    basisname='6-31gs',
    )
printcool('{}'.format(geom))

ref = RHF.from_options(
     geometry= geom, 
     g_convergence=1.0E-6,
     fomo=True,
     fomo_method='gaussian',
     fomo_temp=0.3,
     fomo_nocc=nocc,
     fomo_nact=2,
     print_level=1,
    )
ref.compute_energy()
casci = CASCI.from_options(
    reference=ref,
    nocc=nocc,
    nact=2,
    nalpha=1,
    nbeta=1,
    S_inds=[0],
    S_nstates=[2],
    print_level=1,
    )
casci.compute_energy()
psiw = CASCI_LOT.from_options(
    casci=casci,
    rhf_guess=True,
    rhf_mom=True,
    orbital_coincidence='core',
    state_coincidence='full',
    )

#========================================================#
#|          build the Lightspeed (pyTC) objecs          |#
#========================================================#
#========================================================#
#|                      Geometry:                       |#
#|                      QMMM = False                    |#
#|                      -D3  = False                    |#
#|                      ECP  = False                    |#
#|                                                      |#
#|            Molecule: twisted_fluoroethene            |#
#|               Natom        =           6             |#
#|               Charge       =       0.000             |#
#|               Multiplicity =       1.000             |#
#|                                                      |#
#|                    Basis: 6-31gs                     |#
#|                    nao     =    51                   |#
#|                    ncart   =    51                   

In [4]:
printcool("Build the pyGSM Level of Theory object (LOT)")
lot=PyTC.from_options(states=[(1,0),(1,1)],extra_kwargs={'psiw':psiw},fnm=filepath)

#========================================================#
#|     Build the pyGSM Level of Theory object (LOT)     |#
#========================================================#
 initializing LOT from file


In [5]:
printcool("Build the pyGSM Potential Energy Surface Object (PES)")
pes = PES.from_options(lot=lot,ad_idx=0,multiplicity=1)


#===========================================================#
#|  Build the pyGSM Potential Energy Surface Object (PES)  |#
#===========================================================#


In [6]:
printcool("Build the pyGSM Molecule object \n with Translation and Rotation Internal Coordinates (TRIC)")
M = Molecule.from_options(fnm=filepath,PES=pes,coordinate_type="Cartesian")

#================================================================#
#|               Build the pyGSM Molecule object                |#
#|   with Translation and Rotation Internal Coordinates (TRIC)  |#
#================================================================#
 reading cartesian coordinates from file
 initializing LOT from file


In [7]:
from conjugate_gradient import conjugate_gradient
from _linesearch import backtrack
cg = conjugate_gradient.from_options(Linesearch=backtrack,DMAX=0.25)
#cg = conjugate_gradient.from_options(DMAX=0.1)

In [8]:
geoms,energy = cg.optimize(M,M.energy,'UNCONSTRAINED',30)

 initial E 0.0000
 On opt step 1 
[[-0.108   0.0295 -0.0303 -0.1933  0.0597 -0.4973  0.0117 -0.0946  0.0063
  -0.0257  0.057   0.0006  0.3197  0.0222  0.5918 -0.0044 -0.0738 -0.0711]]
 actual_step= 0.88
 reducing step, new step = 0.25
 Linesearch
 Done linesearch
 dEstep=-27.9384
 Opt step: 1 E: -27.9384 gradrms: 0.20792 ss: 0.250 DMAX: 0.250
 maximum displacement component 0.09 (au)
 maximum gradient component 0.07 (au)

 On opt step 2 
 beta = 0.20
[[-0.1212  0.0234 -0.0703  0.1452  0.0024  0.1469  0.0059 -0.0945  0.0316
  -0.0668  0.0501 -0.013  -0.0118  0.0499  0.0575  0.0487 -0.0314 -0.1526]]
 actual_step= 0.34
 reducing step, new step = 0.25
 Linesearch
 Done linesearch
 dEstep=-16.8680
 Opt step: 2 E: -44.8065 gradrms: 0.09230 ss: 0.250 DMAX: 0.250
 maximum displacement component 0.06 (au)
 maximum gradient component 0.06 (au)

 On opt step 3 
 beta = 0.89
[[ 0.1859  0.0903 -0.0421  0.0197  0.0395  0.1045 -0.0543 -0.112   0.0886
  -0.1519  0.0051 -0.0865 -0.0221  0.076   0.0685 

In [13]:
manage_xyz.write_xyzs_w_comments('cartesian_sd.xyz',geoms,energy,scale=1.)


In [14]:
printcool("Build the pyGSM Molecule object \n with Translation and Rotation Internal Coordinates (TRIC)")
newM = Molecule.from_options(geom = geoms[-1],PES=pes,coordinate_type="TRIC")

#================================================================#
#|               Build the pyGSM Molecule object                |#
#|   with Translation and Rotation Internal Coordinates (TRIC)  |#
#================================================================#
 getting cartesian coordinates from geom
 initializing LOT from file
 making primitives from options!
 making primitive Hessian
 forming Hessian in basis


In [15]:
newM.gradient

array([[-0.    ],
       [-0.    ],
       [ 0.    ],
       [-0.    ],
       [ 0.    ],
       [ 0.0007],
       [-0.    ],
       [ 0.0002],
       [-0.0041],
       [-0.0041],
       [ 0.0057],
       [-0.0005],
       [-0.0031],
       [ 0.0006],
       [ 0.0033],
       [ 0.0016],
       [-0.001 ],
       [-0.    ]])

In [16]:
np.sqrt(np.dot(newM.gradient.T,newM.gradient)/newM.num_coordinates)

array([[ 0.0022]])